# Benzinga-Nachrichten-Verarbeitung

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cwd = "/content/drive/MyDrive/NewsTrading/trading_bot"
%cd /content/drive/MyDrive/NewsTrading/trading_bot

/content/drive/MyDrive/NewsTrading/trading_bot


In [3]:
!sudo apt update
!sudo apt install maven;

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [517 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,017 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,287 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https

In [4]:
%%capture
!pip install html2text
!pip install datefinder
!pip install -U dask[complete]
!pip install nltk
!pip install dateparser
!pip install sutime;

In [5]:
!mvn dependency:copy-dependencies -DoutputDirectory=./jars -f $(python3 -c 'import importlib; import pathlib; print(pathlib.Path(importlib.util.find_spec("sutime").origin).parent / "pom.xml")');


[INFO] Scanning for projects...
[INFO] 
[INFO] ----------< edu.stanford.nlp:stanford-corenlp-sutime-python >-----------
[INFO] Building CoreNLP SUTime Python Wrapper 1.4.0
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] --- maven-dependency-plugin:3.1.2:copy-dependencies (default-cli) @ stanford-corenlp-sutime-python ---
[INFO] javax.xml.bind:jaxb-api:jar:2.4.0-b180830.0359 already exists in destination.
[INFO] javax.activation:javax.activation-api:jar:1.2.0 already exists in destination.
[INFO] com.sun.xml.bind:jaxb-core:jar:2.3.0.1 already exists in destination.
[INFO] com.sun.xml.bind:jaxb-impl:jar:2.4.0-b180830.0438 already exists in destination.
[INFO] javax.activation:activation:jar:1.1.1 already exists in destination.
[INFO] edu.stanford.nlp:stanford-corenlp:jar:4.0.0 already exists in destination.
[INFO] com.apple:AppleJavaExtensions:jar:1.4 already exists in destination.
[INFO] de.jollyday:jollyday:jar:0.4.9 already exists in dest

In [6]:
%load_ext autoreload
%autoreload 2
import dask.dataframe as dd
import dask
import pandas as pd
from dask.distributed import Client
from src.preprocessing.news_parser import filter_body, time, body_formatter, get_company_abbreviation
import re
import plotly.express as px
import nltk
nltk.download('punkt')
import yfinance as yf

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
# client = Client(memory_limit='25GB', processes=False,
#                 n_workers=2, threads_per_worker=1)
# client

## Grobes HTML-Parsing
Als erstes müssen wir die HTML-Dokumente zu normalem Text umwandeln, ansonsten sind die Text-Zellen zu groß und führen zu Problemen mit PyArrow/Dask.

In [ ]:
dask.config.set(scheduler="threads")

In [ ]:
input_dir = "data/raw_bzg/"
output_dir = 'data/unraw1_bzg/'

In [ ]:
# for year in range(2019, 2020):
#     print(year)
#     df = pd.read_parquet(f"{input_dir}story_df_raw_{year}.parquet")
#     df = dd.from_pandas(df, npartitions=12)
#     df["html_body"] = df["html_body"].apply(body_formatter, meta=pd.Series(dtype="str"))
#     df = df.rename(columns={"html_body":"body"})
#     name_function = lambda x: f"data-{year}-{x}.parquet"
#     df.to_parquet(output_dir, name_function=name_function)

## Neu-Partitionierug
Sodass alle Partitionen etwa die gleiche Größe haben.

In [ ]:
input_dir = 'data/unraw1_bzg/'
output_dir = 'data/unraw2_bzg/'

# ddf = dd.read_parquet(input_dir+"*.parquet")
# ddf2 = ddf.repartition(npartitions=50)
# name_function = lambda x: f"data-{x}.parquet"
# ddf2.to_parquet(output_dir, name_function=name_function)

## Author-Inferenz

Ein bisschen die Daten säubern...

In [ ]:
input_dir = cwd+'/data/unraw2_bzg/'
output_dir = cwd+'/data/unraw3_bzg/'

In [ ]:
ddf = dd.read_parquet(input_dir+"*.parquet")

In [ ]:
# Remove rows for which noo stock ticker is recorded
ddf = ddf[ddf.stocks != '']

In [ ]:
# Convert `channels`  datatype from string to list
ddf["channels"] = ddf["channels"].apply(eval, meta=pd.Series(dtype='object'))

Untersuche als nächstes die Behauptung, dass **PRNewswire** und **Businesswire** den gesamten Markt für Pressemeldungen in den USA kontrollieren. Wenn dem so ist, und sie nicht noch weitere, unwichtige Meldungen veröffentlichen, dann können wir einfach die Newsartikel nach diesen Autoren filtern und uns viel Arbeit ersparen.

In [ ]:
dask.config.set(scheduler="processes")
ddf["inferred_author"] = None

def infer_author(body):
  for author in ["PRNewswire", "Globe Newswire", "Business Wire", "ACCESSWIRE"]:
    if re.search(author, body, re.IGNORECASE) is not None:
      return author
  return None

ddf["inferred_author"] = ddf.body.apply(infer_author, meta=pd.Series(dtype="string"))

In [ ]:
# value_counts for authors
auhtor_value_counts = pd.concat([ddf.author.value_counts().head(10), ddf.inferred_author.value_counts().head(10)], axis=1)

In [ ]:
auhtor_value_counts

,author,inferred_author
Benzinga,1061214,NaN
PRNewswire,305720,587242.0
Globe Newswire,293466,475171.0
Business Wire,268561,293052.0
Newsfile,70877,NaN
ACCESSWIRE,62615,81054.0
"AB Digital, Inc.",9936,NaN
WebWire,6404,NaN
PRWeb,2617,NaN
News Direct,2080,NaN


In [ ]:
auhtor_value_counts.sum().diff()

author                  NaN
inferred_author   -646971.0
dtype: float64

Ungefähr 650k Nachrichten werden ausgelassen, wenn nur die vier Hauptvertreiber von Pressemeldungen berücksichtigt werden.

In [ ]:
ddf = ddf[~ddf.inferred_author.isna()]

In [ ]:
ddf["inferred_author"] = ddf["inferred_author"].astype("string")

In [ ]:
ddf["channels"] = ddf.channels.apply(lambda x: str(x), meta=pd.Series(dtype="string"))

In [ ]:
ddf.inferred_author.value_counts().compute()

PRNewswire        587242
Globe Newswire    475171
Business Wire     293052
ACCESSWIRE         81054
Name: inferred_author, dtype: Int64

In [ ]:
ddf.inferred_author.value_counts().sum().compute()

1436519

In [ ]:
name_function = lambda x: f"data-{x}.parquet"
ddf.to_parquet(output_dir, name_function=name_function)

In [ ]:
# Contains 100k rows
earnings_ddf = ddf[ddf.channels.apply(lambda x: "Earnings" in x, meta=pd.Series(dtype=bool))]

In [ ]:
# value counts for authors of earnings reports (contrast to value counts of all news articles)
earnings_ddf.inferred_author.value_counts().head(10)

Globe Newswire    44589
PRNewswire        31440
ACCESSWIRE        16434
Name: inferred_author, dtype: Int64

Hier sehen wir, dass es keine einzige Pressemeldung von **Business Wire** gibt, die mit *Earnings* gekennzeichnet sind. Trotzdem gibt es relevante *Earnings* reports von Business Wire. Dies habe ich kurz verifiziert...

## Full-Name-Discovery, Ticker-Filtering und Ticker-Grouping

Benötigt:
---------
- data/tickers.pkl (alt) \\
- data_shared/corporation_endings.txt \\
- input_dir = data/unraw3_bzg/ \\
- output_dir = data/unraw4_bzg/

Produziert:
-----------
- data_shared/all_ticker_name_mapper.parquet

--------------

Wie viele Nachrichten bleiben, wenn wir auf relevante Ticker filtern? Wir wollen nicht(!) - so ist es momentan - auf die momentane Russell 3k-Zusammensetzung filtern, denn wir wollen auch ungelistete bzw. ehemalige Russell-Aktien beachten.


**1. Full-Name-Discovery:**

Herausfinden des vollen Namens des Unternehmens für jeden Ticker, damit 1. der Text richtig geparst werden kann und 2. damit wir einen Anhaltspunkt für das Ticker-Grouping haben.


**2. Ticker-Filtering:**

Alle Ticker herausfiltern, die wir nicht brauchen. Wenn wir aber ein großes Aktienuniversum (mit inzwischen ungelisteten Aktien) benutzen, werden wir fast alle Nachrichten behalten können. Allerdings lassen sich so Fehlerhafte Nachrichten/Ticker etc. herausfiltern.


**3. Ticker-Grouping:**

Was machen wir, wenn wir mehrerer Aktiengattungen für ein Unternehmen haben? Z.B. Vorzugs- und Stammaktien. Wir können i.A. die Stammaktie nehmen, da diese normalerweise ein höheres Handelsvolumen aufweist. D.h. wir bilden alle Ticker der Benzinga-Nachrichten auf den Ticker der Stammaktie ab.


**4. Firmennamen-Nachrichtenkörper-Verifikation:**

Da Ticker wiederverwendet werden können bzw. sich verändern können wollen wir sicherstellen, dass der Unternehmensname im Nachrichtenkörper vorkommt! Bzw. generell ist das eine gute Datensäuberungs-Maßnahme.

In [8]:
# Get company name by ticker (longName is always equal to shortName in yf...)
# This takes a long time, because of the api calls to yf (15min)
def yahoo_get_wrapper(x):
  try:
    return yf.Ticker(x).info.get("longName")
  except:
    return None

In [9]:
input_dir = cwd+'/data/unraw3_bzg/'
output_dir = cwd+'/data/unraw4_bzg/'
ddf = dd.read_parquet(input_dir)

In [10]:
all_tickers = ddf.stocks.unique().compute()

In [ ]:
# 1. Full-Name-Discovery
company_names = all_tickers.apply(lambda x: yahoo_get_wrapper(x))

In [ ]:
all_mapper = pd.concat([all_tickers, company_names], axis=1)

In [ ]:
all_mapper.columns = ["ticker", "company_name"]

In [ ]:
all_mapper = all_mapper.dropna() # This drops like half of the tickers.

In [ ]:
all_mapper[all_mapper.company_name.apply(lambda x: "Alphabet" in x)]

In [ ]:
print(len(all_mapper))
print(len(all_mapper.company_name.unique()))

In [ ]:
vcs = all_mapper.company_name.value_counts()
vcs = vcs[vcs >= 2]

In [ ]:
# Ticker-Grouping
all_mapper[all_mapper.company_name.isin(vcs.index)].sort_values("company_name")

Es ist nicht leicht zu sagen, welchen von den Tickern wir bevorzugen sollten. Abgleichen mit den Aktientickern des Kursdatensatzes notwendig, um zu sehen, ob überhaupt nur ein Ticker übereinstimmt. Wenn es für beide Ticker eine Kurszeitreihe gibt, dann sollten wir die nehmen, die ein höheres historisches Volumen hat. Dies ist allerdings etwas, was wir später machen und nicht jetzt. Hier wollen wir zunächst nur die Nachrichten verarbeiten, weswegen wir nur die NaN-Unternehmen rausnehmen und den Rest - *ohne Ticker-Filtering* - weiterverarbeiten.

In [ ]:
mapper = all_mapper

In [ ]:
mapper.columns = ["ticker", "company_names"]
mapper = mapper[mapper.isna().sum(axis=1) == 0]

In [ ]:
mapper = mapper.set_index("ticker")

In [ ]:
company_endings = pd.read_table("data_shared/corporation_endings.txt").iloc[:, 0]
mapper["short_name"] = mapper.company_names.apply(lambda x: get_company_abbreviation(x,
                                                                                     company_endings=company_endings))

In [ ]:
print(mapper.short_name.isna().sum()) # 84 stocks for which we don't have an ending to abbreviate
mapper.loc[:, "short_name"] = mapper.short_name.fillna(mapper.company_names)
mapper = mapper.applymap(lambda x: x.strip(" "))

In [ ]:
mapper.to_parquet(cwd + "/data_shared/ticker_name_mapper.parquet")

In [ ]:
mapper = pd.read_parquet(cwd + "/data_shared/ticker_name_mapper.parquet")

In [ ]:
filt_ddf = ddf[ddf.stocks.isin(mapper.index.to_list())]

In [ ]:
# ddf.shape[0].compute()

In [ ]:
# filt_ddf.shape[0].compute()

Es verbleiben circa 660k Nachrichten, die für unser Russell 3000-Aktienuniversum relevant sind. Dieses Filtering hätten wir eigentlich auch schon früher machen können... Aber egal.

In [ ]:
filt_ddf = filt_ddf.set_index("time")

In [ ]:
monthly_news_counts = filt_ddf.stocks.resample("MS").count()

In [ ]:
# px.line(monthly_news_counts.compute(), title="# of articles per month")

In [ ]:
dask.config.set(scheduler="processes")

Diese Nachrichten können wir nun wirklich parsen, und danach ordentlich kategorisieren.

In [ ]:
ddf = filt_ddf
ddf = ddf.drop(columns=["author"]).rename(columns={"inferred_author":"author"})

In [ ]:
# TODO: Kann effizienter werden mit GroupBy-Operation
ddf["company_name"] = ddf.stocks.apply(lambda x: mapper.company_names.loc[x], meta=pd.Series(dtype="string"))
ddf["short_name"] = ddf.stocks.apply(lambda x: mapper.short_name.loc[x], meta=pd.Series(dtype="string"))

In [ ]:
name_function = lambda x: f"data-{x}.parquet"
ddf.to_parquet(cwd+'/data/latest/', name_function=name_function)

## Nachrichten-Parsing


In [ ]:
ddf = dd.read_parquet(cwd+'/data/latest/')

### Firmennamen-Nachrichtenkörper-Verifikation

### Stichproben-Parsing

In [ ]:
sample_partition = ddf.get_partition(40)
x = sample_partition.head(5).iloc[0]
x.body

'_Bank becomes the first to have branches in entire contiguous United States_\n\n_More than halfway through plan to open 400 new branches and hire 3,000\nemployees by the end of 2022_\n\nChase today announced that it has officially opened bank branches in all of\nthe lower 48 states, marking an important milestone in its expansion plans and\nas the first national bank to have a retail presence in the contiguous United\nStates. The bank opened its first branch in Billings, Montana earlier this\nweek following openings in Arkansas, Iowa, Maine, Mississippi, New Mexico,\nNorth and South Dakota, Wyoming and Vermont in July.\n\nThis press release features multimedia. View the full release here:\nhttps://www.businesswire.com/news/home/20210804005697/en/\n\n(Graphic: Business Wire)\n\nThe bank is more than halfway through its plans to open 400 branches in new\nmarkets by the end of 2022. Since announcing expansion plans in 2018, Chase\nhas opened more than 220 branches and entered 25 new stat

In [ ]:
filter_body(x.body, x.stocks, x.author, x.name, x.company_name, x.short_name) # x.name == time

'Bank becomes the first to have branches in entire contiguous United States   More than halfway through plan to open 400 new branches and hire 3,000 employees by the end of 2022  Chase today announced that it has officially opened bank branches in all of the lower 48 states, marking an important milestone in its expansion plans and as the first national bank to have a retail presence in the contiguous United States.The bank opened its first branch in Billings, Montana earlier this week following openings in Arkansas, Iowa, Maine, Mississippi, New Mexico, North and South Dakota, Wyoming and Vermont in July.This press release features multimedia.Since announcing expansion plans in 2018, Chase has opened more than 220 branches and entered 25 new states and the District of Columbia.Approximately 30 percent of these branches will be in low-to- moderate income communities."This is an important milestone for us.We are proud to be able to bring the power of our branches to more households, bus

Anwenden der filter_body-Funktion auf alle Reihen:

In [ ]:
ddf["parsed_body"] = ddf.apply(lambda x: filter_body(x.body,
                                                      x.stocks,
                                                      x.author,
                                                      x.name,
                                                      x.company_name,
                                                      x.short_name),
                                axis=1,
                                meta=pd.Series(dtype="string"))

In [ ]:
ddf = ddf.drop(columns=["body"])

In [ ]:
output_dir = 'data/parsed_bzg/'
name_function = lambda x: f"data-{x}.parquet"
ddf.to_parquet(output_dir, name_function=name_function)

In [ ]:
import requests
import time

while True:
    try:
        requests.get('https://www.google.com')
        print("Kept alive.")
    except:
        print("Failed to keep alive.")
    time.sleep(600)